<a href="https://colab.research.google.com/github/ljkrajewski/ollama-local-rag/blob/main/RAG_colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install ollama and download the tinyllama LLM
!curl -fsSL https://ollama.com/install.sh | sh

import multiprocessing
import os
import time

def run_ollama():
    print(f"Running ollama on PID {os.getpid()}")
    os.system('ollama serve')

ollama_process = multiprocessing.Process(target=run_ollama)
ollama_process.start()
time.sleep(10)

!ollama pull tinyllama
!ollama pull mxbai-embed-large

In [ ]:
#@title Install ollama-local-rag
!pip install python-dotenv
!git clone https://github.com/ljkrajewski/ollama-local-rag.git
%cd ollama-local-rag
ENVFILE="""
LLM_MODEL=tinyllama
EMBEDDING_MODEL=mxbai-embed-large
"""
with open('.env', 'w') as f:
    f.write(ENVFILE)
!pip install -r requirements.txt

import nltk
nltk.download('all')

In [ ]:
#@title Populate the database
from google.colab import files

"""
# Upload files to index
%cd /content/ollama-local-rag/docs
!rm *
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
"""

%cd /content/ollama-local-rag
!python create_database.py

In [ ]:
#@title Start gradio
!pip install -q gradio

import gradio as gr
import re
import subprocess
import sys

def generate(prompt):
    """
    Runs a separate Python script and returns its output.

    Args:
        script_path (str): The path to the Python script to execute.
        *args: Any arguments to pass to the script.

    Returns:
        tuple: A tuple containing the script's standard output (stdout) and standard error (stderr) as strings,
               or None if an error occurred.
    """
    script_path = "/content/ollama-local-rag/query_data.py"
    args = [prompt]
    try:
        # Construct the command to run the script
        command = [sys.executable, script_path] + list(args)

        # Run the script and capture its output
        process = subprocess.run(command, capture_output=True, text=True, check=True)

        # Return the stdout and stderr
        return process.stdout, process.stderr

    except subprocess.CalledProcessError as e:
        # Handle errors from the subprocess
        print(f"Error running script: {e}")
        return e.stdout, e.stderr #Return the error output if the subprocess failed.

    except FileNotFoundError:
        print(f"Script not found: {script_path}")
        return None, None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None
  return result

with gr.Blocks(analytics_enabled=False) as demo:
  with gr.Column():
    with gr.Row():
      prompt_box = gr.Textbox(lines=10, interactive=True, value="", label="Prompt")
      with gr.Column():
        generate_button = gr.Button("Generate")
        log_box = gr.Textbox(lines=1, interactive=False, label="System Messages")
    result_box = gr.Markdown(show_copy_button=True, label="Result", show_label=True, container=True, line_breaks=True, height="1000")
  generate_button.click(fn=generate, inputs=[prompt_box], outputs=[result_box, log_box])

demo.queue().launch(inline=False, share=True, debug=True)

In [ ]:
"""
#@title Start xterm
#@markdown To query: ```python query_data.py "<your question here>"```
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm
%xterm
"""